<a href="https://colab.research.google.com/github/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION/blob/main/Algoritmos_isaac_ropero_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico
## Isaac Ropero Mendez
### https://github.com/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION


In [16]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/03MIAR_Algoritmos_de_optimizacion/proyecto/datos_problemas.csv')

df.index += 1
df.drop(df.columns[0], axis='columns', inplace= True)
df.drop('Unnamed: 11', axis='columns', inplace= True)
df.drop('Total', axis='columns', inplace= True)

display(df)

,1,2,3,4,5,6,7,8,9,10
1,1,1,1,1,1,0,0,0,0,0
2,0,0,1,1,1,0,0,0,0,0
3,0,1,0,0,1,0,1,0,0,0
4,1,1,0,0,0,0,1,1,0,0
5,0,1,0,1,0,0,0,1,0,0
6,1,1,0,1,1,0,0,0,0,0
7,1,1,0,1,1,0,0,0,0,0
8,1,1,0,0,0,1,0,0,0,0
9,1,1,0,1,0,0,0,0,0,0
10,1,1,0,0,0,1,0,0,1,0


### Objetivos
Reducir coste total. </br>
30 tomas. </br>
10 actores. </br>
No mas de 6 tomas por dia. </br>
Los actores cobran lo mismo por dias independientemente del numero de tomas.</br>



### Algoritmo voraz


#### Funciones auxiliares

In [26]:
import random

def asignar_grupos(df, n=6):
  #grupos = ['Rodaje ' + str(i) for i in range(1, n)]
  df['Grupo'] = (df.index - 1) // n + 1
  return df

def calcular_coste(df):
  grouped = df.groupby('Grupo').sum()
  coste = (grouped != 0).sum(axis=1)
  return coste

def swap_rows(df):
    i, j = random.sample(range(df.shape[0]), 2)
    df.iloc[i], df.iloc[j] = df.iloc[j].copy(), df.iloc[i].copy()
    return df

def get_actors(df, scene):
  return df.iloc[[scene]].sum(axis=1).values[0]

#### Algoritmo voraz

In [36]:
df_voraz = df.copy()
numero_escenas_dia = 6

# Ordenar por numero de unos
df_voraz['actors'] = df_voraz.sum(axis=1)
sorted_df = df_voraz.sort_values(by='actors', ascending=False)
sorted_df = sorted_df.reset_index(drop=True)
sorted_df = sorted_df.drop(columns=['actors'])
sorted_df.index += 1
sorted_df
df_voraz = asignar_grupos(sorted_df, numero_escenas_dia)

df_original = df.copy()
asignar_grupos(df_original, numero_escenas_dia)

print(calcular_coste(df_original), calcular_coste(df_original).sum())
print(calcular_coste(df_voraz), calcular_coste(df_voraz).sum())

Grupo
1    7
2    8
3    8
4    7
5    8
dtype: int64 38
Grupo
1    9
2    7
3    8
4    7
5    5
dtype: int64 36


### Busqueda aleatoria


In [41]:
# Busqueda aleatoria

def busqueda_aleatoria(df, n, max_intentos=1000):
  mejor_coste = float('inf')
  mejor_solucion = df
  sin_mejora = 0

  for i in range(n):
    if sin_mejora >= max_intentos:
            break

    aux = swap_rows(df)
    asignar_grupos(aux)
    coste = calcular_coste(aux)

    if coste.sum() < mejor_coste:
      mejor_coste = coste.sum()
      mejor_solucion = aux.copy()
      sin_mejora = 0
    else:
      sin_mejora += 1

  return mejor_solucion

In [38]:
df_aleatoria = df_voraz.copy()
asignar_grupos(df_aleatoria)

print(calcular_coste(df_aleatoria), calcular_coste(df_aleatoria).sum())
sol = busqueda_aleatoria(df_aleatoria, 5000)
print(calcular_coste(sol), calcular_coste(sol).sum())

# Probar cambiando el numero de dias, en vez de 30/6, probar diferentes numeros.

Grupo
1    9
2    7
3    8
4    7
5    5
dtype: int64 36
Grupo
1    5
2    8
3    6
4    7
5    7
dtype: int64 33


### Hill-climbing


In [50]:
def hill_climbing(df, n=1000, dias = 6):
    solucion_actual = df.copy()
    coste_actual = calcular_coste(solucion_actual).sum()

    for _ in range(n):
        # Generar una solucion
        posible_solucion = swap_rows(solucion_actual.copy())
        asignar_grupos(posible_solucion)

        # Calcular el coste de la solucion
        posible_solucion_coste = calcular_coste(posible_solucion).sum()

        # Si la solucion es mejor, actualizar el coste
        if posible_solucion_coste < coste_actual:
            solucion_actual, coste_actual = posible_solucion, posible_solucion_coste

    return solucion_actual



In [56]:
dias = 6
sol_actual = []


for x in range(3, dias):
  df_hill_climbing = df_voraz.copy()
  asignar_grupos(df_hill_climbing, x)
  coste = calcular_coste(df_hill_climbing).sum()

  sol = hill_climbing(df_hill_climbing, 7000, x)
  coste_sol_sum = calcular_coste(sol).sum()
  #print(calcular_coste(sol), coste_sol_sum)

  if coste_sol_sum < coste:
    sol_actual = sol

display(sol_actual)
print(calcular_coste(sol_actual), calcular_coste(sol_actual).sum())


,1,2,3,4,5,6,7,8,9,10,Grupo
1,1,1,1,1,1,0,0,0,0,0,1
2,1,1,1,0,1,0,0,1,0,0,1
3,1,1,1,1,0,1,0,0,0,0,1
4,0,0,1,1,1,0,0,0,0,0,1
5,1,0,0,0,1,1,0,0,0,0,1
6,1,1,1,1,0,0,0,0,0,0,1
7,1,0,1,0,1,0,0,0,1,0,2
8,1,0,1,1,1,0,0,0,0,0,2
9,1,1,0,1,0,0,0,0,0,1,2
10,1,1,0,1,1,0,0,0,0,0,2


Grupo
1    7
2    7
3    6
4    6
5    3
dtype: int64 29
